Computations of the BSD parameters: Tamagawa product, Omega and special value of the L-function for CN curves with n upto 1,000,000

In [ ]:
from sage.all import *
from sage.parallel.decorate import parallel
import csv
import itertools
import time

INPUT_FILE = 'data_with_mod8.csv'
OUTPUT_FILE = "bsd_sqf1mil_new.csv"
CHUNK_SIZE = 150


@parallel(ncpus=4)  # Adjust ncpus to match your machine
def process_rows_parallel(chunk):
    results = []
    for row in chunk:
        try:
            n = Integer(row[0])
            if n < 322061:
                continue
            ran = Integer(row[2])
            #if not is_squarefree(n):
                #continue

            a4 = -(n)**2  # Already minimal for squarefree n
            E = EllipticCurve([0, 0, 0, a4, 0])
            #con = E.conductor()
            tam = E.tamagawa_product()
            omg = E.period_lattice().omega()
            spl = E.lseries().dokchitser().derivative(1,ran)/ran.factorial()
            tor = 4
            results.append(row[0:] + [tam] + [tor] + [omg] + [spl])
        except Exception as e:
            print(f"Error processing n={row[0]}: {e}")
    return results

def stream_process_csv():
    with open(INPUT_FILE, newline='') as infile:
        reader = csv.reader(infile)
        header = next(reader)
        data_iter = iter(reader)

        # Initialize output file with header if it doesn't exist
        if not os.path.exists(OUTPUT_FILE):
            with open(OUTPUT_FILE, 'w', newline='') as outfile:
                writer = csv.writer(outfile)
                head_new = header + ['tamagawa'] + ['torsion'] +['omega'] + ['special']
                writer.writerow(head_new)

        while True:
            chunk = list(itertools.islice(data_iter, CHUNK_SIZE))
            if not chunk:
                break

            for _, result in process_rows_parallel([chunk]):
                with open(OUTPUT_FILE, 'a', newline='') as outfile:
                    writer = csv.writer(outfile)
                    for row in result:
                        writer.writerow(row)

# Run main processing
stream_process_csv()
